# Cortex Analyst Endpoint

In [ ]:
from snowflake.snowpark.context import get_active_session
import _snowflake
import json

session = get_active_session()
DATABASE = "CORTEX_ANALYST_DEMO"
SCHEMA = "JAFFLE"
STAGE = "ANALYST"
SEMANTIC_MODEL_FILENAME = "jaffle_shop_star_schema.yaml"

## Make API call and examine results

In [ ]:
prompt = "What is the total count of orders?"
request_body = {
    "messages": [
        {
            "role": "user",
            "content": [{"type": "text", "text": prompt}],
        }
    ],
    "semantic_model_file": f"@{DATABASE}.{SCHEMA}.{STAGE}/{SEMANTIC_MODEL_FILENAME}",
}

api_response = _snowflake.send_snow_api_request(
    "POST",
    "/api/v2/cortex/analyst/message",
    {},
    {},
    request_body,
    {},
    30000,
)

if api_response.get("status", 500) < 400:
    response_content = json.loads(api_response["content"])
    print("Parsed Response:", response_content)
else:
    raise Exception(f"Failed request with status {api_response['status']}: {api_response}")

## Examine Cortex Analyst API response

In [ ]:
content = json.loads(api_response['content'])
analyst_response = content['message']
print(analyst_response)

### Examine the last response - SQL

In [ ]:
content = json.loads(api_response['content'])
analyst_response = content['message']['content'][-1]
print(analyst_response)

## Extract SQL statement for the Cortex Analyst response and execute

In [ ]:
# Parse the API response
content = json.loads(api_response['content'])

# Extract the analyst message
# Note: use "message" (singular) as in the API response.
analyst_response = content['message']

# Verify the structure to avoid KeyError
if 'content' not in analyst_response:
    raise KeyError(f"Expected key 'content' not found in analyst_response. Available keys: {list(analyst_response.keys())}")

# Extract the SQL statement
sql_statement = ""
for item in analyst_response['content']:
    if item.get('type') == 'sql':
        sql_statement = item.get('statement', "")
        # Remove trailing semicolon if present
        if sql_statement.endswith(';'):
            sql_statement = sql_statement[:-1]
        break  # Assuming you only need the first SQL statement

print(sql_statement)

#### Execute SQL

In [ ]:
session.sql(sql_statement)